**Install Kaggle components**

In [ ]:
!pip3 -q install kaggle

In [ ]:
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/kaggle/kaggle.json

chmod: cannot access '/root/kaggle/kaggle.json': No such file or directory


**Download and Unzip Dataset**

In [ ]:
!kaggle datasets download -d paultimothymooney/breast-histopathology-images

100% 3.09G/3.10G [00:23<00:00, 183MB/s]
100% 3.10G/3.10G [00:23<00:00, 144MB/s]


In [ ]:
!unzip -o -q 'breast-histopathology-images.zip'

In [ ]:
import os
from os.path import isfile

**Create new Directory**

In [ ]:
# Create a new directory to store all available images
all_images_dir = 'all_images_dir'
os.mkdir(all_images_dir)

# Create Positive subdirectory within all images
os.mkdir('all_images_dir/1')
os.mkdir('all_images_dir/0')

In [ ]:
import shutil

**Develop dataset**

In [ ]:
# copies all images from their seperate folders into the same 
# folder called all_images_dir


# create a list of all patient id's
# each patient id folder has 2 sub folders --> folder 0 and folder 1

# Example:
    # '10285'
        # '0'
        # '1'

cur_dir = os.getcwd()
patient_list = os.listdir(cur_dir)

for patient in patient_list:
  
  try:
    
    path_0 = str(patient) + '/0'
    path_1 = str(patient) + '/1'

    # create a list of all files in folder 0
    file_list_0 = os.listdir(path_0)
    # create a list of list all file in folder 1
    file_list_1 = os.listdir(path_1)

    # move the 0 images to all_images_dir
    for fname in file_list_0:

        # source path to image
        src = os.path.join(path_0, fname)
        # destination path to image
        dst = os.path.join(all_images_dir, '0', fname)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)

    # move the 1 images to all_images_dir
    for fname in file_list_1:

        # source path to image
        src = os.path.join(path_1, fname)
        # destination path to image
        dst = os.path.join(all_images_dir, '1', fname)
        # copy the image from the source to the destination
        shutil.copyfile(src, dst)
        
  except:
     print(patient, 'cannot be copied into proper folder')

.config cannot be copied into proper folder
all_images_dir cannot be copied into proper folder
breast-histopathology-images.zip cannot be copied into proper folder
IDC_regular_ps50_idx5 cannot be copied into proper folder
sample_data cannot be copied into proper folder


In [ ]:
import os

In [ ]:
# Check how many images are in each folder

print('Total:', len(os.listdir('all_images_dir/1'))+len(os.listdir('all_images_dir/0')))
print('Positive:', len(os.listdir('all_images_dir/1')))
print('Negative:', len(os.listdir('all_images_dir/0')))

Total: 277524
Positive: 78786
Negative: 198738


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = '/content/all_images_dir/0/10253_idx5_x1001_y1001_class0.png'
img = image.load_img(img_path, target_size = (50,50))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
#x = preprocess_input(x)

f = model.predict(x)
print(type(f))
f.shape

<class 'numpy.ndarray'>


(1, 1, 1, 512)

**Preprocess image and Extract features for Train**

In [ ]:
import numpy as np
import cv2
import os
from scipy import ndimage
from scipy.spatial import distance
from sklearn.cluster import KMeans
from google.colab.patches import cv2_imshow

# Return a dictionary that holds all images category by category. 
def load_images_from_folder(folder):
    images = {}
    ct = 0
    sift_vectors = {}
    descriptor_list = []
    sift = cv2.xfeatures2d.SIFT_create()
    for filename in os.listdir(folder):
        category = []
        path = folder + "/" + filename
        for cat in os.listdir(path)[:10000]:
          ct += 1
          img = cv2.imread(path + "/" + cat,0)
          img = cv2.GaussianBlur(img, (5, 5), 0)
          ret, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
          if img is not None:
            kp, des = sift.detectAndCompute(img, None)
            
            pt = str('/content/all_images_dir/' + filename + '/'+ cat)
            imag = image.load_img(pt, target_size = (50,50))
            x = image.img_to_array(imag)
            x = np.expand_dims(x, axis=0)
            ft = model.predict(x)
            g = ft.reshape(4,128)

            #sg = des.reshape(1920)
            try:
              bfeat = np.vstack((g, des))
              category.append(bfeat)
              descriptor_list.extend(bfeat)
              print(str(ct) + " ---> " +str(bfeat.shape))
            except:
              continue
        sift_vectors[filename] = category
    #cv2_imshow(img)
    #print('image ->' + cat)
    print(ct)
    return [sift_vectors, descriptor_list]

all_feat_train = load_images_from_folder('all_images_dir')[0]
descriptor_list = load_images_from_folder('all_images_dir')[1]  # take all train images category by category 
#test = load_images_from_folder("BreastCancer/test") # take test images 

Streaming output truncated to the last 5000 lines.
14853 ---> (5, 128)
14854 ---> (40, 128)
14855 ---> (40, 128)
14856 ---> (40, 128)
14857 ---> (12, 128)
14858 ---> (17, 128)
14860 ---> (6, 128)
14861 ---> (8, 128)
14862 ---> (49, 128)
14863 ---> (61, 128)
14864 ---> (69, 128)
14865 ---> (13, 128)
14866 ---> (13, 128)
14867 ---> (21, 128)
14868 ---> (63, 128)
14869 ---> (34, 128)
14870 ---> (13, 128)
14871 ---> (6, 128)
14872 ---> (30, 128)
14873 ---> (9, 128)
14874 ---> (27, 128)
14875 ---> (41, 128)
14876 ---> (32, 128)
14877 ---> (10, 128)
14878 ---> (44, 128)
14879 ---> (34, 128)
14880 ---> (25, 128)
14881 ---> (8, 128)
14882 ---> (21, 128)
14883 ---> (40, 128)
14884 ---> (18, 128)
14885 ---> (25, 128)
14886 ---> (40, 128)
14887 ---> (29, 128)
14888 ---> (31, 128)
14889 ---> (12, 128)
14890 ---> (39, 128)
14891 ---> (19, 128)
14892 ---> (18, 128)
14893 ---> (10, 128)
14894 ---> (28, 128)
14895 ---> (11, 128)
14896 ---> (6, 128)
14897 ---> (17, 128)
14899 ---> (28, 128)
14900 ---> 

In [ ]:
descriptor_list = []

for key,value in all_feat_train.items():
        for img in value:
          descriptor_list.extend(img)


In [ ]:
len(descriptor_list)

744251

In [ ]:
from tensorflow.keras.preprocessing import image
img_path = '/content/all_images_dir/0/10253_idx5_x1001_y1001_class0.png'
img = image.load_img(img_path, target_size = (50,50))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
# x = preprocess_input(x)
f = model.predict(x)
g = f.reshape(4,128)
g.shape
#f.shape

(4, 128)

In [ ]:
import cv2

img_path = '/content/all_images_dir/0/10253_idx5_x1001_y1001_class0.png'
img = cv2.imread(img_path,0)
sift = cv2.xfeatures2d.SIFT_create()
kp, des = sift.detectAndCompute(img, None)
des.shape

(15, 128)

In [ ]:
import cv2

img_path = '/content/10253/0/10253_idx5_x2201_y1101_class0.png'
img = cv2.imread(img_path,0)
sift = cv2.xfeatures2d.SIFT_create()
kp, des = sift.detectAndCompute(img, None)
des.shape

(31, 128)

In [ ]:
bothfeat = np.vstack((g, des))
bothfeat.shape

(35, 128)

**Preprocess image and Extract features for Test**

In [ ]:
import numpy as np
import cv2
import os
from scipy import ndimage
from scipy.spatial import distance
from sklearn.cluster import KMeans
from google.colab.patches import cv2_imshow

# Return a dictionary that holds all preprocessed images features category by category. 
def load_images_from_folder(folder):
    images = {}
    ct = 0
    sift_vectors = {}
    descriptor_list = []
    sift = cv2.xfeatures2d.SIFT_create()
    for filename in os.listdir(folder):
        category = []
        path = folder + "/" + filename
        for cat in os.listdir(path)[10000:12500]:
          ct += 1
          img = cv2.imread(path + "/" + cat,0)
          img = cv2.GaussianBlur(img, (5, 5), 0)
          ret, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
          if img is not None:
            kp, des = sift.detectAndCompute(img, None)
            
            pt = str('/content/all_images_dir/' + '/' + filename + '/'+ cat)
            imag = image.load_img(pt, target_size = (50,50))
            x = image.img_to_array(imag)
            x = np.expand_dims(x, axis=0)
            ft = model.predict(x)
            g = ft.reshape(4,128)

            try:
              bfeat = np.vstack((g, des))
              category.append(bfeat)
              print(str(ct) + " ---> " +str(bfeat.shape))
            except:
              continue
        sift_vectors[filename] = category

    print(ct)
    return sift_vectors


all_feat_test = load_images_from_folder('all_images_dir')  # take all train images category by category 
#test = load_images_from_folder("BreastCancer/test") # take test images 

1 ---> (15, 128)
2 ---> (17, 128)
3 ---> (89, 128)
4 ---> (56, 128)
5 ---> (49, 128)
6 ---> (79, 128)
7 ---> (89, 128)
8 ---> (54, 128)
9 ---> (18, 128)
10 ---> (12, 128)
11 ---> (43, 128)
12 ---> (37, 128)
13 ---> (39, 128)
14 ---> (59, 128)
15 ---> (57, 128)
16 ---> (49, 128)
17 ---> (24, 128)
18 ---> (7, 128)
19 ---> (89, 128)
20 ---> (58, 128)
21 ---> (77, 128)
22 ---> (64, 128)
23 ---> (34, 128)
24 ---> (51, 128)
25 ---> (92, 128)
26 ---> (46, 128)
27 ---> (28, 128)
28 ---> (32, 128)
29 ---> (5, 128)
30 ---> (54, 128)
31 ---> (13, 128)
32 ---> (50, 128)
33 ---> (72, 128)
34 ---> (15, 128)
36 ---> (55, 128)
37 ---> (45, 128)
38 ---> (38, 128)
39 ---> (76, 128)
40 ---> (41, 128)
41 ---> (59, 128)
42 ---> (53, 128)
43 ---> (57, 128)
44 ---> (9, 128)
45 ---> (28, 128)
46 ---> (33, 128)
47 ---> (7, 128)
49 ---> (8, 128)
50 ---> (45, 128)
51 ---> (13, 128)
52 ---> (30, 128)
53 ---> (72, 128)
54 ---> (53, 128)
55 ---> (34, 128)
56 ---> (20, 128)
57 ---> (59, 128)
58 ---> (23, 128)
59 ---

In [ ]:
len(all_feat_train['1'])

9860

In [ ]:
len(all_feat_train['0'])

9729

In [ ]:
len(all_feat_test['0'])

2431

In [ ]:
len(all_feat_test['1'])

2462

In [ ]:
import pickle
dictionary_data = {"Bothfeattrain":all_feat_train, "Bothfeattest":all_feat_test}

a_file = open("20KBothFeatures.pkl", "wb")
pickle.dump(dictionary_data, a_file)
a_file.close()

In [ ]:
!pip install opencv-python==3.4.2.17
!pip install opencv-contrib-python==3.4.2.17

     |████████████████████████████████| 25.0 MB 2.9 MB/s 
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 30.6 MB 1.4 MB/s 
  Attempting uninstall: opencv-contrib-python
    Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30


In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
model = VGG19(weights='imagenet', include_top=False)

80150528/80134624 [==============================] - 0s 0us/step


In [ ]:
import numpy as np
import cv2
import os
from scipy import ndimage
from scipy.spatial import distance
from sklearn.cluster import KMeans
from google.colab.patches import cv2_imshow

In [ ]:
(imlen(sifts[0])

665895

In [ ]:
len(sifts[1]['0'])

9729

In [ ]:
type(sifts[1])

dict

In [ ]:
import pickle
dictionary_data = {"Sifttrain":sift_train, "Sifttest":sift_test}

a_file = open("20KSiftFeatures.pkl", "wb")
pickle.dump(dictionary_data, a_file)
a_file.close()

In [ ]:
len(sift_train['0'])

10000

In [ ]:
len(sift_test['1'])

NameError: ignored

In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount GDrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

Mounted at /content/drive


In [ ]:
ls

20kAllfeat90Histog.pkl  20kAllfeatHistog.pkl  20KBothFeatures.pkl


In [ ]:
cd '/content/drive/MyDrive/32ABC'


/content/drive/MyDrive/32ABC


In [ ]:
import pickle
with open('20KBothFeatures.pkl', 'rb') as f:
    all_feat_train = pickle.load(f)['Bothfeattrain']


In [ ]:
import pickle
with open('20KBothFeatures.pkl', 'rb') as f:
    all_feat_test = pickle.load(f)['Bothfeattest']

**Create Disciptor list for Clustering**

In [ ]:
descriptor_list = []

for key,value in all_feat_train.items():
        for img in value:
          descriptor_list.extend(img)

In [ ]:
len(descriptor_list)

744251

**K means clustering**

In [ ]:
from sklearn.cluster import KMeans

def kmeans(k, descriptor_list):
    kmeans = KMeans(n_clusters = k, n_init=10)
    kmeans.fit(descriptor_list)
    visual_words = kmeans.cluster_centers_ 
    return visual_words
    
# Takes the central points which is visual words    
visual_words = kmeans(120, descriptor_list)

In [ ]:
len(visual_words)

120

**Histogram generation**

In [ ]:
import numpy as np
from scipy.spatial import distance
def find_index(image, center):
    count = 0
    ind = 0
    for i in range(len(center)):
        if(i == 0):
           count = distance.euclidean(image, center[i]) 
           #count = L1_dist(image, center[i])
        else:
            dist = distance.euclidean(image, center[i]) 
            #dist = L1_dist(image, center[i])
            if(dist < count):
                ind = i
                count = dist
    return ind

def image_class(all_bovw, centers):
    dict_feature = {}
    ct = 0
    for key,value in all_bovw.items():
        category = []
        for img in value:
            histogram = np.zeros(len(centers))
            ct +=1
            print(ct)
            for each_feature in img:
                ind = find_index(each_feature, centers)
                histogram[ind] += 1
            category.append(histogram)
        dict_feature[key] = category
    return dict_feature
    
# Creates histograms for train data    
bovw_both_train = image_class(all_feat_train, visual_words) 
# Creates histograms for test data

Streaming output truncated to the last 5000 lines.
14590
14591
14592
14593
14594
14595
14596
14597
14598
14599
14600
14601
14602
14603
14604
14605
14606
14607
14608
14609
14610
14611
14612
14613
14614
14615
14616
14617
14618
14619
14620
14621
14622
14623
14624
14625
14626
14627
14628
14629
14630
14631
14632
14633
14634
14635
14636
14637
14638
14639
14640
14641
14642
14643
14644
14645
14646
14647
14648
14649
14650
14651
14652
14653
14654
14655
14656
14657
14658
14659
14660
14661
14662
14663
14664
14665
14666
14667
14668
14669
14670
14671
14672
14673
14674
14675
14676
14677
14678
14679
14680
14681
14682
14683
14684
14685
14686
14687
14688
14689
14690
14691
14692
14693
14694
14695
14696
14697
14698
14699
14700
14701
14702
14703
14704
14705
14706
14707
14708
14709
14710
14711
14712
14713
14714
14715
14716
14717
14718
14719
14720
14721
14722
14723
14724
14725
14726
14727
14728
14729
14730
14731
14732
14733
14734
14735
14736
14737
14738
14739
14740
14741
14742
14743
14744
14745
14746
14747
1

In [ ]:
bovw_both_test = image_class(all_feat_test, visual_words) 

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
import pickle
dictionary_data = {"AllfHtrain":bovw_both_train, "AllfHtest":bovw_both_test}

a_file = open("20kAllfeat120Histog.pkl", "wb")
pickle.dump(dictionary_data, a_file)
a_file.close()

In [ ]:
len(all_bovw_feature['0'])

9729

In [ ]:
len(test_bovw_feature['0'])

2431

In [ ]:
len(bovw_s_train['0'])

9729

In [ ]:
import pickle
dictionary_data = {"siftrain":bovw_s_train, "siftest":bovw_s_test}

a_file = open("20KSifFilesss.pkl", "wb")
pickle.dump(dictionary_data, a_file)
a_file.close()

In [ ]:
import pickle
with open('20KFilesss.pkl', 'rb') as f:
    bovw_test = pickle.load(f)['test']

with open('20KFilesss.pkl', 'rb') as f:
    bovw_train = pickle.load(f)['train']

with open('20KFilesss.pkl', 'rb') as f:
    svm_test_labels = pickle.load(f)['test_labels']

with open('20KFilesss.pkl', 'rb') as f:
    svm_labels = pickle.load(f)['labels']

with open('20KSifFilesss.pkl', 'rb') as f:
    bovw_s_test = pickle.load(f)['siftest']

with open('20KSifFilesss.pkl', 'rb') as f:
    bovw_s_train = pickle.load(f)['siftrain']

In [ ]:
type(bovw_train)

numpy.ndarray

In [ ]:
len(bovw_train)

20000

In [ ]:
type(bovw_s_train)

dict

In [ ]:
len(bovw_s_train['0'])

9729

In [ ]:
len(bovw_s_train['1'])

9860

In [ ]:
import numpy as np
bovwstack_both_train = np.array([np.zeros(90)])

for key,value in bovw_both_train.items():
  category = []
  for img in value:
    bovwstack_both_train = np.vstack([bovwstack_both_train, img])

In [ ]:
len(bovwstack_both_train)

In [ ]:
import numpy as np
bovwstack_both_test = np.array([np.zeros(90)])

for key,value in bovw_both_test.items():
  category = []
  for img in value:
    bovwstack_both_test = np.vstack([bovwstack_both_test, img])

In [ ]:
len(bovwstack_both_test)

In [ ]:
import numpy as np
bovwstack_both_test = np.delete(bovwstack_both_test, 0, 0)
bovwstack_both_train = np.delete(bovwstack_both_train, 0,0)

In [ ]:
print('train ->' + str(len(bovwstack_both_train)) +'   test ->' + str(len(bovwstack_both_test )))

In [ ]:
train_labels = np.array(['1'])
for i in range(9859):
  train_labels = np.append(train_labels,'1')
for i in range(9729):
  train_labels = np.append(train_labels, '0')
train_labels.shape

In [ ]:
test_labels = np.array(['1'])
for i in range(2461):
  test_labels = np.append(test_labels,'1')
for i in range(2431):
  test_labels = np.append(test_labels, '0')
test_labels.shape

In [ ]:
import pickle
dictionary_data = {"train":bovwstack_both_train, "test":bovwstack_both_test, "test_labels":test_labels, "train_labels":train_labels}

a_file = open("90kFilesss.pkl", "wb")
pickle.dump(dictionary_data, a_file)
a_file.close()

In [ ]:
import pickle
with open('70kFilesss.pkl', 'rb') as f:
    bovwstack_both_test = pickle.load(f)['test']

with open('70kFilesss.pkl', 'rb') as f:
    bovwstack_both_train = pickle.load(f)['train']

with open('70kFilesss.pkl', 'rb') as f:
    test_labels = pickle.load(f)['test_labels']

with open('70kFilesss.pkl', 'rb') as f:
    train_labels = pickle.load(f)['train_labels']

gamma 0.0008 -> 0.66.56

gamma 0.005 -> 0.66.8

gamma 0.006 -> 0.67

gamma 0.007 -> 0.67.15

**gamma 0.008 -> 0.67.19**

gamma 0.009 -> 0.67.05

gamma 0.09 -> 0.65.86

C 1.24 -> 67.11

**C 1.34 -> 67.23**

C 1.44 -> 67.19

C 1.54 -> 67.17

C 1.64 -> 67.07

In [ ]:
from sklearn import svm
from sklearn.metrics import confusion_matrix
print("Applying SVM classifier.")
gm = 0.008
cc = 1.34
svm_model = svm.SVC(kernel='rbf', random_state = 1, gamma = gm, C = cc)
svm_model.fit(bovwstack_both_train, train_labels)
#print("Gamma - > {}".format(gm))
#print("C - > {}".format(cc))
y_pred = svm_model.predict(bovwstack_both_test)

svm_model.score(bovwstack_both_test, test_labels)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(test_labels, y_pred)
print(cm)

In [ ]:
print(classification_report(test_labels, y_pred))

In [ ]:
import seaborn as sns
sns.heatmap(cm, annot=True,fmt='d')